In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data=pd.read_csv('Train_Data_Final.csv')
eval_data=pd.read_csv('Test_Data.csv')
eval_data.pop('Unnamed: 0')
data.pop('id')
eval_data.pop('id')

data['Date'] = pd.to_datetime(data['Date'])
data['year'] = data['Date'].dt.year
data['month'] = data['Date'].dt.month
eval_data['Date'] = pd.to_datetime(eval_data['Date'])
eval_data['year'] = eval_data['Date'].dt.year
eval_data['month'] = eval_data['Date'].dt.month

data['Time'] = pd.to_datetime(data['Time'])
data['hour'] = data['Time'].dt.strftime('%H')
eval_data['Time'] = pd.to_datetime(eval_data['Time'])
eval_data['hour'] = eval_data['Time'].dt.strftime('%H')

data.pop('Date')
data.pop('Time')
data.pop('year')
eval_data.pop('Date')
eval_data.pop('Time')
eval_data.pop('year')



# Use list comprehension to replace values in the original list
mapping = {'Dry':0, 'Ideal':1, 'Slightly Elevated':2,'Elevated':3, 'High':4}
transformed_list = [mapping[val] if val in mapping else val for val in data['RH_type']]
data['RH_type']=transformed_list.copy()


y=data.pop('RH_type')
x=data
x_eval=eval_data


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [5]:
from sklearn.linear_model import LogisticRegression


model = LogisticRegression(solver='newton-cg', multi_class='multinomial', random_state=0)
H = model.fit(x_train, y_train)

#print('Logistic Regression Model Coeff (m) =\n' , model.coef_)
#print('\nLogistic Regression Model inteceps (b) =\n' , model.intercept_)


# Predicting the data
y_predict=model.predict(x_test)
y_predict_pro=model.predict_proba(x_test)
# Predicting of testing data that is final csv submition
y_predict_newdata=model.predict(x_eval)

# Evaluate the Model
# print("\nPrediction Probability : \n",y_predict_pro)
#print("\nPrediction : \n",y_predict)


/home/mypc/.local/lib/python3.11/site-packages/sklearn/utils/optimize.py:246: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


In [ ]:

mapping = {0:'Dry',1: 'Ideal',2: 'Slightly Elevated',3:'Elevated',4: 'High'}
transformed_list1 = [mapping[val] if val in mapping else val for val in y_predict_newdata]

y_predict_newdata=transformed_list1.copy()
y_predict_newdata=pd.DataFrame(y_predict_newdata)

y_predict_newdata.to_csv("y_predict.csv",index=True)


In [6]:
from sklearn.metrics import roc_auc_score, accuracy_score ,f1_score,confusion_matrix

def calculate_metrics(y_true, y_pred_proba, y_pred):
    """Calculate AUC-ROC, accuracy, and F1 score"""
    auc_roc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # Assuming one-vs-rest strategy
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')  # weighted average for multiclass
    return auc_roc, accuracy, f1


auc_roc, accuracy, f1 = calculate_metrics(y_test, y_predict_pro, y_predict)
print("AUC-ROC:", auc_roc)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Confusion Matrix : \n",confusion_matrix(y_test, model.predict(x_test)))

AUC-ROC: 0.96027534194405
Accuracy: 0.80375
F1 Score: 0.8021659241767045
Confusion Matrix : 
 [[305  30   0   0   0]
 [ 18 446  30   0   1]
 [  0  59 216  42   2]
 [  0   1  48 166  43]
 [  0   0   4  36 153]]
